In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('netflix_titles.csv')

In [3]:
df.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


In [4]:
def create_textual_representation(row):
    textual_data = f"""Type : {row['type']},
Title : {row['title']},
Director: {row['director']},
Cast: {row['cast']},
Released: {row['release_year']},
Genre: {row['listed_in']},
Description: {row['description']}"""
    
    return textual_data

In [5]:
df['textual_representation'] = df.apply(create_textual_representation,axis=1)

In [6]:
df['textual_representation']

0       Type : Movie,\nTitle : Dick Johnson Is Dead,\n...
1       Type : TV Show,\nTitle : Blood & Water,\nDirec...
2       Type : TV Show,\nTitle : Ganglands,\nDirector:...
3       Type : TV Show,\nTitle : Jailbirds New Orleans...
4       Type : TV Show,\nTitle : Kota Factory,\nDirect...
                              ...                        
8802    Type : Movie,\nTitle : Zodiac,\nDirector: Davi...
8803    Type : TV Show,\nTitle : Zombie Dumb,\nDirecto...
8804    Type : Movie,\nTitle : Zombieland,\nDirector: ...
8805    Type : Movie,\nTitle : Zoom,\nDirector: Peter ...
8806    Type : Movie,\nTitle : Zubaan,\nDirector: Moze...
Name: textual_representation, Length: 8807, dtype: object

In [9]:
df['textual_representation'].values[0]

'Type : Movie,\nTitle : Dick Johnson Is Dead,\nDirector: Kirsten Johnson,\nCast: nan,\nReleased: 2020,\nGenre: Documentaries,\nDescription: As her father nears the end of his life, filmmaker Kirsten Johnson stages his death in inventive and comical ways to help them both face the inevitable.'

In [10]:
import faiss
import requests

In [11]:
dim =4096
index = faiss.IndexFlatL2(dim)

X = np.zeros((len(df['textual_representation']), dim), dtype='float32')

In [15]:
for i, representation in enumerate(df['textual_representation']):
    if i%1000 ==0:
        print("response submitted .,",i)


    res = requests.post("http://localhost:11434/api/embeddings",
                        json = {
                        'model': 'llama2',
                        'prompt': representation,
                        }
                        )
    
    embedding = res.json()['embedding']
    X[i] = np.array(embedding)

index.add(X)


response submitted ., 0


KeyboardInterrupt: 

In [ ]:
faiss.write_index(index,'index')

In [ ]:
index = faiss.read_index('index')

In [13]:
df[df.title.str.contains('Shutter')]

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,textual_representation
1358,s1359,Movie,Shutter Island,Martin Scorsese,"Leonardo DiCaprio, Mark Ruffalo, Ben Kingsley,...",United States,"February 1, 2021",2010,R,139 min,Thrillers,A U.S. marshal's troubling visions compromise ...,"Type : Movie,\nTitle : Shutter Island,\nDirect..."
8013,s8014,Movie,Shutter,"Banjong Pisanthanakun, Parkpoom Wongpoom","Ananda Everingham, Natthaweeranuch Thongmee, A...",Thailand,"September 5, 2018",2004,TV-MA,96 min,"Horror Movies, International Movies",After killing a young girl in a hit-and-run ac...,"Type : Movie,\nTitle : Shutter,\nDirector: Ban..."


In [14]:
favourite_movie = df.iloc[1358]

In [ ]:
res = requests.post("http://localhost:11434/api/embeddings",
                        json = {
                        'model': 'llama2',
                        'prompt': favourite_movie['textual_rpresentation'],
                        }
                        )

In [ ]:
embedding = np.array([res.json()['embedding']],dtype='float32')

In [ ]:
D, I = index.search(embedding,5)

In [ ]:
best_matches = np.array(df['textual_representation'])[I.flatten()]

In [ ]:
for match in best_matches:
    print('NEXT MOVIE')
    print(match)
    print()